In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [9]:
!kaggle datasets download -d jangedoo/utkface-new

100% 330M/331M [00:15<00:00, 24.8MB/s]
100% 331M/331M [00:15<00:00, 22.5MB/s]


In [10]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [11]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [12]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [13]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [14]:
len(age)

23708

In [15]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [16]:
df.head()

,age,gender,img
0,46,0,46_0_0_20170104211749300.jpg.chip.jpg
1,59,1,59_1_0_20170110123116580.jpg.chip.jpg
2,52,0,52_0_4_20170104212325060.jpg.chip.jpg
3,84,1,84_1_0_20170120230257434.jpg.chip.jpg
4,51,1,51_1_0_20170109010116440.jpg.chip.jpg


In [17]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [18]:
train_df.shape

(20000, 3)

In [19]:
test_df.shape

(3708, 3)

In [20]:
train_datagen = ImageDataGenerator(
                        rescale=1./255,
                        rotation_range=30,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [21]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [22]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [23]:
resnet = ResNet50(include_top=False,input_shape=(200,200,3))

94765736/94765736 [==============================] - 5s 0us/step


In [24]:
resnet.trainable = False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512,activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)


output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [25]:
model = Model(inputs=resnet.input, outputs=[output1, output2])

In [26]:
model.compile(optimizer='adam', loss={"age":'mae', 'gender':'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [27]:
model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)

Epoch 1/10
625/625 [==============================] - 267s 409ms/step - loss: 97.2207 - age_loss: 15.4784 - gender_loss: 0.8257 - age_mae: 15.4784 - gender_accuracy: 0.5145 - val_loss: 83.6355 - val_age_loss: 15.0172 - val_gender_loss: 0.6931 - val_age_mae: 15.0172 - val_gender_accuracy: 0.5178
Epoch 2/10
625/625 [==============================] - 251s 401ms/step - loss: 83.6783 - age_loss: 15.1101 - gender_loss: 0.6926 - age_mae: 15.1101 - gender_accuracy: 0.5235 - val_loss: 83.0624 - val_age_loss: 14.5054 - val_gender_loss: 0.6925 - val_age_mae: 14.5054 - val_gender_accuracy: 0.5181
Epoch 3/10
625/625 [==============================] - 259s 415ms/step - loss: 83.8523 - age_loss: 14.9045 - gender_loss: 0.6964 - age_mae: 14.9045 - gender_accuracy: 0.5230 - val_loss: 83.0012 - val_age_loss: 14.4394 - val_gender_loss: 0.6925 - val_age_mae: 14.4394 - val_gender_accuracy: 0.5178
Epoch 4/10
625/625 [==============================] - 249s 399ms/step - loss: 83.4372 - age_loss: 14.9096 - gend